---
## Simple generator example reading midi files
### Monday: 26-02-2024

* Read the MIDI file normally
    * We are given a list with all midi elements

* Read the MIDI file using the generator 
    * We are "`yield`ed" each element individually
    * We can get elements with `next()` or with a `loop`

---

In [7]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from handlers.midiHandler import MidiHandler
from utils.Utils import Utils as utils

midi_file_path = "../data/midi/test.mid"
midi = MidiHandler()

# Read the MIDI file normally
data = midi.readMidi(midi_file_path)
print(data[:5]) # Print 5 to save notebook space

# Read the MIDI file using the generator 
data = midi.readMidiGenerator(midi_file_path)
count = 0
for d in data:
    count += 1
    print(d)
    if count > 5: # Print 5 to save notebook space
        break


[MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0), MetaMessage('set_tempo', tempo=500000, time=0), MetaMessage('track_name', name='Tempo Track', time=0), MetaMessage('track_name', name='New Instrument', time=0), Message('note_on', channel=0, note=92, velocity=25, time=0.7265625)]
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('track_name', name='Tempo Track', time=0)
MetaMessage('track_name', name='New Instrument', time=0)
note_on channel=0 note=92 velocity=25 time=0.7265625
control_change channel=0 control=64 value=29 time=0.09895833333333333


---
## Lets get some random Midi sections
### Friday: 01-03-2024

* Read the MIDI file and retrieve the notes we want
   * Handle 1 file with and without generators

* Read the MIDI file using generators and threads
    * Use all cpu cores with yield generators
    
* Compare the differences between using generators and not
---

In [8]:
from concurrent.futures import ThreadPoolExecutor

directory = "../data/midi/pop/"
file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]

def getNotes(midi_file_path):
    midi = MidiHandler()
    notes = midi.getNotes(midi_file_path)
    #print(notes)
    print("Total notes retrieved: ", len(notes))

def getNotesGenerator(midi_file_path):
    midi = MidiHandler()
    notes = midi.getNotesGenerator(midi_file_path)
    count = 0
    for n in notes:
        #print(n)
        count += 1
    print("Total notes retrieved using generator: ", count)

def getNotesThreaded(directory):
    file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]
    num_workers = min(os.cpu_count(), len(file_paths))
    print("Number of workers: ", num_workers)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(midi.getNotes, file_path) for file_path in file_paths]
        count = 0
        for future in futures:
            if future.result() is not None:
                notes = future.result()
                #print(notes)
                count += len(notes)
    print("Total notes retrieved using threaded: ", count)

def getNotesThreadedGenerator(directory):
    file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]
    num_workers = min(os.cpu_count(), len(file_paths)) 
    print("Number of workers: ", num_workers)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(midi.getNotesGenerator, file_path) for file_path in file_paths]
        count = 0
        for future in futures:
            notes = future.result()
            if notes is not None:
                for n in notes:
                    #print(n)
                    count += 1
    print("Total notes retrieved using threaded generator: ", count)

print("\n"+"--Using normal function")
utils.measure_function(getNotes, midi_file_path)

print("\n"+"--Using generator function")
utils.measure_function(getNotesGenerator, midi_file_path)

print("Total MIDI files: ", len(file_paths))

print("\n"+"--Using threaded function")
utils.measure_function(getNotesThreaded, directory)

print("\n"+"--Using threaded generator function")
utils.measure_function(getNotesThreadedGenerator, directory)



--Using normal function
Total notes retrieved:  5
Memory used: 0.00390625 MB
Time taken: 0.0009999275207519531 seconds


--Using generator function
Total notes retrieved using generator:  5
Memory used: 0.0 MB
Time taken: 0.0010039806365966797 seconds

Total MIDI files:  2622

--Using threaded function
Number of workers:  20
Total notes retrieved using threaded:  1510
Memory used: 54.359375 MB
Time taken: 292.4176013469696 seconds


--Using threaded generator function
Number of workers:  20
Total notes retrieved using threaded generator:  1510
Memory used: -9.10546875 MB
Time taken: 184.87527418136597 seconds

